Script para criação do arquivo chave.txt - usado para lançamento de cte >Especificos Mafra>Rotinas>Fretes

In [ ]:
# SCRIPT IN PYTHON BY DINO GREJO - DINOGREJO@GMAIL.COM GITHUB: DGREJ

import os
import re

# Pasta contendo os arquivos XML
caminho_pasta = r'C:\Users\dino.grejo\OneDrive - VIVEO\Documentos\python\xml'

# Caminho para salvar o arquivo chave.txt
caminho_arquivo_chave = r'C:\Users\dino.grejo\OneDrive - VIVEO\Documentos\python\chave.txt'

# Regex para extrair os números da tag chave
regex = r'<chCTe>(\d+)</chCTe>'

# Abrindo o arquivo de texto para escrita
with open(caminho_arquivo_chave, 'w') as arquivo_chave:
    # Escrevendo "CHAVE" no cabeçalho do arquivo TXT
    arquivo_chave.write('CHAVE\n')
    
    # Iterando sobre cada arquivo na pasta
    for nome_arquivo in os.listdir(caminho_pasta):
        if nome_arquivo.endswith('.xml'):
            # Abrindo o arquivo XML e lendo o conteúdo
            with open(os.path.join(caminho_pasta, nome_arquivo), 'r') as arquivo_xml:
                conteudo = arquivo_xml.read()
                # Extraindo os números da tag chave usando regex
                match = re.search(regex, conteudo)
                if match:
                    numero_chave = match.group(1)
                    # Escrevendo o número da chave no arquivo de texto
                    arquivo_chave.write(numero_chave + ",1" + '\n')

print(f"Arquivo chave.txt gerado em: {caminho_arquivo_chave}")


Script para criação de planilha para lançamento de CT-e FOB (HL) >Atualizações>Movimentos>Nt Conhec. Frete

In [ ]:
#SCRIPT IN PYTHON BY DINO GREJO - DINOGREJO@GMAIL.COM GITHUB: DGREJ

import os
import xml.etree.ElementTree as ET
import pandas as pd

# Pasta contendo os arquivos XML
caminho_pasta = r'C:\Users\dino.grejo\OneDrive - VIVEO\Documentos\python\xml'

# Lista para armazenar os dados extraídos
dados = []

# Função para extrair as informações de cada XML
def extrair_dados_do_xml(caminho_xml):
    tree = ET.parse(caminho_xml)
    root = tree.getroot()
    
    # Definir o namespace
    ns = {'nfe': 'http://www.portalfiscal.inf.br/cte'}
    
    # Função auxiliar para extrair texto de elementos com verificação
    def get_element_text(path):
        element = root.find(path, ns)
        return element.text if element is not None else ''
    
    # Ajuste para remover os dois primeiros dígitos
    def ajustar_codigo_municipio(codigo):
        return codigo[2:] if len(codigo) > 2 else codigo

    # Extrair as informações desejadas
    cte_info = {
        'CNPJ FILIAL': get_element_text('.//nfe:dest/nfe:CNPJ'),
        'CNPJ TRANSP.':get_element_text('.//nfe:emit/nfe:CNPJ'),
        'TRANSPORTADOR':get_element_text('.//nfe:emit/nfe:xNome'),
        'REMETENTE': get_element_text('.//nfe:rem/nfe:xNome'),
        'CNPJ_REMETENTE': get_element_text('.//nfe:rem/nfe:CNPJ'),
        'CTE': get_element_text('.//nfe:ide/nfe:nCT'),
        'SERIE': get_element_text('.//nfe:ide/nfe:serie'),
        'VALOR': get_element_text('.//nfe:vPrest/nfe:vTPrest').replace('.', ','),
        'CHAVEnf': '\n'.join([chave.text for chave in root.findall('.//nfe:infDoc/nfe:infNFe/nfe:chave', ns)]),
        'EMISSAO': get_element_text('.//nfe:ide/nfe:dhEmi').split('T')[0] if get_element_text('.//nfe:ide/nfe:dhEmi') else '',  # Formato de data apenas
        'UFIni': get_element_text('.//nfe:ide/nfe:UFIni'),
        'cMunIni': ajustar_codigo_municipio(get_element_text('.//nfe:ide/nfe:cMunIni')),
        'xMunIni': get_element_text('.//nfe:ide/nfe:xMunIni'),
        'UFFim': get_element_text('.//nfe:ide/nfe:UFFim'),
        'cMunFim': ajustar_codigo_municipio(get_element_text('.//nfe:ide/nfe:cMunFim')),
        'xMunFim': get_element_text('.//nfe:ide/nfe:xMunFim'),
        'chCTe': get_element_text('.//nfe:infProt/nfe:chCTe'),
        'vTrib': get_element_text('.//nfe:vPrest/nfe:vTPrest').replace('.', ','),
        'icms%': get_element_text('.//nfe:ICMS/nfe:ICMS00/nfe:pICMS'),
        'vICMS': get_element_text('.//nfe:ICMS/nfe:ICMS00/nfe:vICMS').replace('.', ','),
        'ICMSOutUF%': get_element_text('.//nfe:ICMS/nfe:ICMSOutraUF/nfe:pICMSOutraUF'),
        'vICMSOutUF': get_element_text('.//nfe:ICMS/nfe:ICMSOutraUF/nfe:vICMSOutraUF').replace('.', ','),
        
    }

     # Ajuste para a data de emissão no formato desejado (dd/mm/aaaa)
    if cte_info['EMISSAO']:
        data_emissao = cte_info['EMISSAO'].split('-')
        cte_info['EMISSAO'] = f"{data_emissao[2]}/{data_emissao[1]}/{data_emissao[0]}"
    
    return cte_info

# Percorrer todos os arquivos XML na pasta
for arquivo in os.listdir(caminho_pasta):
    if arquivo.endswith('.xml'):
        caminho_xml = os.path.join(caminho_pasta, arquivo)
        dados.append(extrair_dados_do_xml(caminho_xml))

# Criar um DataFrame com os dados extraídos
df = pd.DataFrame(dados)

# Adicionar colunas vazias (em branco) ao DataFrame
colunas_extras = ['Venc.', 'Data lanç.', 'serviço', 'NOME FILIAL', 'COD']
for coluna in colunas_extras:
    df[coluna] = ''  # Preenche a coluna com valores vazios

# Ajustar a ordem das colunas de acordo com o modelo Excel
df = df[['Venc.', 'Data lanç.', 'serviço', 'CNPJ FILIAL', 'NOME FILIAL', 'CNPJ TRANSP.', 'COD', 'TRANSPORTADOR', 'REMETENTE', 'CNPJ_REMETENTE', 'CTE', 'SERIE', 'VALOR', 'CHAVEnf', 'EMISSAO', 'UFIni', 'cMunIni', 'xMunIni', 'UFFim', 
         'cMunFim', 'xMunFim', 'chCTe', 'vTrib', 'icms%', 'vICMS', 'ICMSOutUF%', 'vICMSOutUF']]

# Gerar a planilha Excel
caminho_excel = r'C:\Users\dino.grejo\OneDrive - VIVEO\Documentos\python\Fatura_CTes_FOB.xlsx'
df.to_excel(caminho_excel, index=False)

print(f'Planilha gerada em: {caminho_excel}')


Script para criação de planilha para lançamento de CT-e CIF - >Atualizações>Movimentos>Nt Conhec. Frete

In [ ]:
#SCRIPT IN PYTHON BY DINO GREJO - DINOGREJO@GMAIL.COM GITHUB: DGREJ

import os
import xml.etree.ElementTree as ET
import pandas as pd

# Pasta contendo os arquivos XML
caminho_pasta = r'C:\Users\dino.grejo\OneDrive - VIVEO\Documentos\python\xml'

# Lista para armazenar os dados extraídos
dados = []

# Função para extrair as informações de cada XML
def extrair_dados_do_xml(caminho_xml):
    tree = ET.parse(caminho_xml)
    root = tree.getroot()
    
    # Definir o namespace
    ns = {'nfe': 'http://www.portalfiscal.inf.br/cte'}
    
    # Função auxiliar para extrair texto de elementos com verificação
    def get_element_text(path):
        element = root.find(path, ns)
        return element.text if element is not None else ''
    
    # Ajuste para remover os dois primeiros dígitos
    def ajustar_codigo_municipio(codigo):
        return codigo[2:] if len(codigo) > 2 else codigo

    # Extrair as informações desejadas
    cte_info = {
        'CNPJ REMETENTE': get_element_text('.//nfe:rem/nfe:CNPJ'),
        'REMETENTE': get_element_text('.//nfe:rem/nfe:xNome'),
        'CNPJ TRANSP.':get_element_text('.//nfe:emit/nfe:CNPJ'),
        'TRANSPORTADOR':get_element_text('.//nfe:emit/nfe:xNome'),
        'DESTINATARIO': get_element_text('.//nfe:dest/nfe:xNome'),
        'CNPJ DESTINATARIO': get_element_text('.//nfe:dest/nfe:CNPJ'),
        'CTE': get_element_text('.//nfe:ide/nfe:nCT'),
        'SERIE': get_element_text('.//nfe:ide/nfe:serie'),
        'VALOR': get_element_text('.//nfe:vPrest/nfe:vTPrest').replace('.', ','),
        'CHAVEnf': '\n'.join([chave.text for chave in root.findall('.//nfe:infDoc/nfe:infNFe/nfe:chave', ns)]),
        'EMISSAO': get_element_text('.//nfe:ide/nfe:dhEmi').split('T')[0] if get_element_text('.//nfe:ide/nfe:dhEmi') else '',  # Formato de data apenas
        'UFIni': get_element_text('.//nfe:ide/nfe:UFIni'),
        'cMunIni': ajustar_codigo_municipio(get_element_text('.//nfe:ide/nfe:cMunIni')),
        'xMunIni': get_element_text('.//nfe:ide/nfe:xMunIni'),
        'UFFim': get_element_text('.//nfe:ide/nfe:UFFim'),
        'cMunFim': ajustar_codigo_municipio(get_element_text('.//nfe:ide/nfe:cMunFim')),
        'xMunFim': get_element_text('.//nfe:ide/nfe:xMunFim'),
        'chCTe': get_element_text('.//nfe:infProt/nfe:chCTe'),
        'vTrib': get_element_text('.//nfe:vPrest/nfe:vTPrest').replace('.', ','),
        'icms%': get_element_text('.//nfe:ICMS/nfe:ICMS00/nfe:pICMS'),
        'vICMS': get_element_text('.//nfe:ICMS/nfe:ICMS00/nfe:vICMS').replace('.', ','),
        'ICMSOutUF%': get_element_text('.//nfe:ICMS/nfe:ICMSOutraUF/nfe:pICMSOutraUF'),
        'vICMSOutUF': get_element_text('.//nfe:ICMS/nfe:ICMSOutraUF/nfe:vICMSOutraUF').replace('.', ','),
        
    }

     # Ajuste para a data de emissão no formato desejado (dd/mm/aaaa)
    if cte_info['EMISSAO']:
        data_emissao = cte_info['EMISSAO'].split('-')
        cte_info['EMISSAO'] = f"{data_emissao[2]}/{data_emissao[1]}/{data_emissao[0]}"
    
    return cte_info

# Percorrer todos os arquivos XML na pasta
for arquivo in os.listdir(caminho_pasta):
    if arquivo.endswith('.xml'):
        caminho_xml = os.path.join(caminho_pasta, arquivo)
        dados.append(extrair_dados_do_xml(caminho_xml))

# Criar um DataFrame com os dados extraídos
df = pd.DataFrame(dados)

# Adicionar colunas vazias (em branco) ao DataFrame
colunas_extras = ['Venc.', 'Data lanç.', 'FATURA','serviço', 'COD']
for coluna in colunas_extras:
    df[coluna] = ''  # Preenche a coluna com valores vazios

# Ajustar a ordem das colunas de acordo com o modelo Excel
df = df[['Venc.', 'Data lanç.', 'FATURA','serviço', 'CNPJ REMETENTE', 'REMETENTE', 'CNPJ TRANSP.', 'COD', 'TRANSPORTADOR', 'DESTINATARIO', 'CNPJ DESTINATARIO', 'CTE', 'SERIE', 'VALOR', 'CHAVEnf', 'EMISSAO', 'UFIni', 'cMunIni', 'xMunIni', 'UFFim', 
         'cMunFim', 'xMunFim', 'chCTe', 'vTrib', 'icms%', 'vICMS', 'ICMSOutUF%', 'vICMSOutUF']]

# Gerar a planilha Excel
caminho_excel = r'C:\Users\dino.grejo\OneDrive - VIVEO\Documentos\python\Fatura_CTes_CIF.xlsx'
df.to_excel(caminho_excel, index=False)

print(f'Planilha gerada em: {caminho_excel}')


CODE AJUSTADO FOB HL - INSERIR VIA PROMPR VENCIMENTO E FILIAL

In [ ]:
# SCRIPT IN PYTHON BY DINO GREJO - DINOGREJO@GMAIL.COM GITHUB: DGREJ

import os
import xml.etree.ElementTree as ET
import pandas as pd

# Pasta contendo os arquivos XML
caminho_pasta = r'C:\Users\dino.grejo\OneDrive - VIVEO\Documentos\python\xml'

# Solicitar valores via prompt
vencimento = input("Digite a data de vencimento para a coluna 'Venc.': ")
nome_filial_numero = input("Digite o número para a 'NOME FILIAL' (ex: 001002): ")

# Lista para armazenar os dados extraídos
dados = []

# Função para extrair as informações de cada XML
def extrair_dados_do_xml(caminho_xml):
    tree = ET.parse(caminho_xml)
    root = tree.getroot()
    
    # Definir o namespace
    ns = {'nfe': 'http://www.portalfiscal.inf.br/cte'}
    
    # Função auxiliar para extrair texto de elementos com verificação
    def get_element_text(path):
        element = root.find(path, ns)
        return element.text if element is not None else ''
    
    # Ajuste para remover os dois primeiros dígitos
    def ajustar_codigo_municipio(codigo):
        return codigo[2:] if len(codigo) > 2 else codigo

    # Extrair as informações desejadas
    cte_info = {
        'CNPJ FILIAL': get_element_text('.//nfe:dest/nfe:CNPJ'),
        'CNPJ TRANSP.': get_element_text('.//nfe:emit/nfe:CNPJ'),
        'TRANSPORTADOR': get_element_text('.//nfe:emit/nfe:xNome'),
        'REMETENTE': get_element_text('.//nfe:rem/nfe:xNome'),
        'CNPJ_REMETENTE': get_element_text('.//nfe:rem/nfe:CNPJ'),
        'CTE': get_element_text('.//nfe:ide/nfe:nCT'),
        'SERIE': get_element_text('.//nfe:ide/nfe:serie'),
        'VALOR': get_element_text('.//nfe:vPrest/nfe:vTPrest').replace('.', ','),
        'CHAVEnf': '\n'.join([chave.text for chave in root.findall('.//nfe:infDoc/nfe:infNFe/nfe:chave', ns)]),
        'EMISSAO': get_element_text('.//nfe:ide/nfe:dhEmi').split('T')[0] if get_element_text('.//nfe:ide/nfe:dhEmi') else '',  # Formato de data apenas
        'UFIni': get_element_text('.//nfe:ide/nfe:UFIni'),
        'cMunIni': ajustar_codigo_municipio(get_element_text('.//nfe:ide/nfe:cMunIni')),
        'xMunIni': get_element_text('.//nfe:ide/nfe:xMunIni'),
        'UFFim': get_element_text('.//nfe:ide/nfe:UFFim'),
        'cMunFim': ajustar_codigo_municipio(get_element_text('.//nfe:ide/nfe:cMunFim')),
        'xMunFim': get_element_text('.//nfe:ide/nfe:xMunFim'),
        'chCTe': get_element_text('.//nfe:infProt/nfe:chCTe'),
        'vTrib': get_element_text('.//nfe:vPrest/nfe:vTPrest').replace('.', ','),
        'icms%': get_element_text('.//nfe:ICMS/nfe:ICMS00/nfe:pICMS'),
        'vICMS': get_element_text('.//nfe:ICMS/nfe:ICMS00/nfe:vICMS').replace('.', ','),
        'ICMSOutUF%': get_element_text('.//nfe:ICMS/nfe:ICMSOutraUF/nfe:pICMSOutraUF'),
        'vICMSOutUF': get_element_text('.//nfe:ICMS/nfe:ICMSOutraUF/nfe:vICMSOutraUF').replace('.', ',')
    }

    # Ajuste para a data de emissão no formato desejado (dd/mm/aaaa)
    if cte_info['EMISSAO']:
        data_emissao = cte_info['EMISSAO'].split('-')
        cte_info['EMISSAO'] = f"{data_emissao[2]}/{data_emissao[1]}/{data_emissao[0]}"
    
    return cte_info

# Percorrer todos os arquivos XML na pasta
for arquivo in os.listdir(caminho_pasta):
    if arquivo.endswith('.xml'):
        caminho_xml = os.path.join(caminho_pasta, arquivo)
        cte_info = extrair_dados_do_xml(caminho_xml)
        
        # Adicionar os valores fixos e inseridos via prompt
        cte_info['Venc.'] = vencimento
        cte_info['serviço'] = "FOB"
        cte_info['NOME FILIAL'] = f"CM {nome_filial_numero}"
        cte_info['COD'] = "000003"
        cte_info['Data lanç.'] = ""  # Deixa a coluna vazia para preenchimento manual posteriormente
        cte_info['cod_Rem']= r"PROCV([@[CNPJ_REMETENTE]];'C:\Users\dino.grejo\OneDrive - VIVEO\Documentos\!FOB CM HL\[cod_fornecedores.xlsx]fornecedores'!$B$1:$D$420;2;FALSO" #cod procV excel - extrair informação da planilha cod_fornecedores
        cte_info['loj_Rem']= r"PROCV([@[CNPJ_REMETENTE]];'C:\Users\dino.grejo\OneDrive - VIVEO\Documentos\!FOB CM HL\[cod_fornecedores.xlsx]fornecedores'!$B$1:$D$420;3;FALSO" #cod procV excel - extrair informação da planilha cod_fornecedores
        
        # Adicionar o dicionário de dados à lista
        dados.append(cte_info)

# Criar um DataFrame com os dados extraídos
df = pd.DataFrame(dados)

# Ajustar a ordem das colunas de acordo com o modelo Excel
df = df[['Venc.', 'Data lanç.', 'serviço', 'CNPJ FILIAL', 'NOME FILIAL', 'CNPJ TRANSP.', 'COD', 'TRANSPORTADOR', 
         'REMETENTE', 'CNPJ_REMETENTE','cod_Rem','loj_Rem','CTE', 'SERIE', 'VALOR', 'CHAVEnf', 'EMISSAO', 'UFIni', 'cMunIni', 
         'xMunIni', 'UFFim', 'cMunFim', 'xMunFim', 'chCTe', 'vTrib', 'icms%', 'vICMS', 'ICMSOutUF%', 'vICMSOutUF']]

# Gerar a planilha Excel
caminho_excel = r'C:\Users\dino.grejo\OneDrive - VIVEO\Documentos\python\Fatura_CTes_FOB.xlsx'
df.to_excel(caminho_excel, index=False)

print(f'Planilha gerada em: {caminho_excel}')


SCRIP PARA JUNTAR AQUIVOS EXCEL COM O MESMO FORMATO DE COLUNAS

In [ ]:
import pandas as pd

# Defina o diretório onde os arquivos estão localizados
diretorio_entradas = r'C:\Users\dino.grejo\OneDrive - VIVEO\Documentos\python\Tabelas_excel'

# Defina os nomes dos arquivos .xlsx
# Coloque os nomes dos arquivos aqui
tabelas = ['1.xlsx', '2.xlsx', '3.xlsx', '4.xlsx', '5.xlsx', '6.xlsx', '7.xlsx', '8.xlsx', '9.xlsx']
           

# Crie uma lista para armazenar as tabelas
dfs = []

# Leia cada tabela e adicione à lista
for tabela in tabelas:
    df = pd.read_excel(f'{diretorio_entradas}/{tabela}')
    dfs.append(df)

# Concatene as tabelas
tabela_conc = pd.concat(dfs, ignore_index=True)

# Defina o diretório de saída
# Coloque o local onde o arquivo será salvo
diretorio_saida =  r'C:\Users\dino.grejo\OneDrive - VIVEO\Documentos\python'

# Salve a tabela juntada em um arquivo Excel
tabela_conc.to_excel(f'{diretorio_saida}/tabela_conc.xlsx', index=False)

# Confirme que o arquivo foi salvo
print(f'Arquivo salvo em: {diretorio_saida}/tabela_conc.xlsx')